# ConvNet regression micro-bead tracker
## Find the position of one micro-bead in the image 
Using DL regression based on convnet + densely connected layers.
The training, validation and test-data are generated by simulations.

In [ ]:
%matplotlib inline
#%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import holodeep2D

# physics parameters for the simulations:
n_m  = 1.339        # refractive index medium
n_p  = 1.59         # index sphere (polystirene: 1.55-1.59):
lamb = 0.660        # illumination wavelength (microns)
umpp = 0.300        # camera micron (um) per pixel

In [ ]:
def make_training_test_set(num_imgs=5000):
    ''' return and save the training set, by making 'num_imgs' hologram images. '''    
    # parameters (modify them):
    img_dim_px = [20,20]
    savename = '' #'./holodeep_trainset'
    img_scale = .7
    bead_rad_um = 0.5
    bead_zmin_um = -3.3
    bead_zmax_um = -3.3
    bead_noise = 0.05
    # init images:
    imgs = np.zeros((num_imgs, img_dim_px[0], img_dim_px[1]))
    # init data labels:
    pos  = np.zeros((num_imgs, 3))                                                                                          
    # make rand holograms images:
    for img_i in range(num_imgs):
        print('creating image: '+str(img_i)+' /'+str(num_imgs), end='\r')
        # make bead hologram:
        d_i = holodeep2D.make_bead_holo(img_dim_px=img_dim_px, img_scale=img_scale, bead_rad_um=bead_rad_um, zmin_um=bead_zmin_um, zmax_um=bead_zmax_um, noise=bead_noise, plots=False)
        imgs[img_i,:,:] = d_i['holo_bead']
        pos[img_i,:]    = d_i['holo_bead_pos_px']
    # normalize the image data to mean 0 and std 1. 
    X = (imgs - np.mean(imgs)) / np.std(imgs)
    # normalize labels (x,y,z) so x,y are in 0,1:
    Y = pos/np.array((img_dim_px[0], img_dim_px[1], 1)) + np.array((0.5, 0.5, 0))
    Y = Y[:,:2]
    if savename:
        np.save(savename, {'X':X, 'Y':Y})
        print('saved: '+savename+'.npy')
    return X, Y

Create by simulation training images (X) and labels (Y).
Y gives the 2D position of the bead in the image

In [ ]:
X,Y = make_training_test_set(num_imgs=5000)

Visualize one image with its label:

In [ ]:
plt.imshow(X[0])
plt.title(Y[0])

Define here the network. Change the hyper parameters to make it better.

In [ ]:
def define_model():
    ''' define and compile the model. 
    input_shape : include the channel as in [img_dim_px[0], img_dim_px[1], 1]
    label_shape : 3 or 2
    '''
    from keras.models import Sequential
    from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Activation, Dropout, BatchNormalization
    from keras import optimizers 
    from keras import regularizers
    label_shape = 2
    input_shape = (20,20,1)
    model = Sequential()
    # define model architecture:
    model.add(Conv2D(32, (3,3), activation='relu', padding='valid', input_shape=input_shape))
    model.add(Conv2D(64, (3,3), activation='relu', padding='valid', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(128, (3,3), activation='relu', padding='valid'))
    model.add(Conv2D(256, (3,3), activation='relu', padding='valid'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(256, activation='relu')) 
    model.add(Dense(128, activation='relu')) 
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(label_shape, activation=None))
    # compile model:
    model.compile(loss='mean_squared_error', optimizer='adamax', metrics=['accuracy'])
    return model


def plot_model(model):
    ''' plot training history '''
    loss = model.history.history['loss']
    acc  = model.history.history['acc']
    val_loss = model.history.history['val_loss']
    val_acc  = model.history.history['val_acc']
    epochs = model.history.epoch
    fig = plt.figure('train_model()', clear=False)
    ax1 = fig.add_subplot(211)
    ax2 = fig.add_subplot(212)
    ax1.semilogy(epochs, loss, label='loss')
    ax1.semilogy(epochs, val_loss, label='val_loss')
    ax1.legend()
    ax2.plot(epochs, acc, label='acc') 
    ax2.plot(epochs, val_acc, label='val_acc') 
    ax2.set_xlabel('epochs')
    ax2.legend()

    
model = define_model()

Train the model here, and plot its history:

In [ ]:
def train_model(model, X, Y, epochs=10):
    ''' returns trained model '''
    history = model.fit(x=X[:,:,:,np.newaxis], y=Y, epochs=epochs, batch_size=16, verbose=1, validation_split=0.2)
    plot_model(model)
    return model

model = train_model(model, X,Y, epochs=4)

Test the model on a test set which resembles the experiment
(the bead moves along a circular trajectory of diameter 'ampl' pixels)

In [ ]:
def test_model(model, nimgs=100, ampl=3):
    ''' a test set for the trained model'''
    X = np.zeros((nimgs, 20, 20, 1))
    Y = np.zeros((nimgs, 2))
    w = 0.1
    for i in range(nimgs):
        print(f'creating test set {i} / {nimgs}', end='\r')
        x = ampl*np.sin(w*i)   
        y = ampl*np.cos(w*i)  
        z = -3.3/umpp #(- 6.5 - i/nimgs)/umpp
        d = holodeep2D.make_bead_holo(xyzpos_px=[x,y,z])
        X[i,:,:,0] = d['holo_bead']
        Y[i,:] = d['holo_bead_pos_px'][:2]
    # normalize:
    X = (X - np.mean(X))/np.std(X)
    Y = Y/20 + 0.5
    # predict:
    pred = model.predict(X)
    # error (distance predictions - truth):
    err_rel = np.hypot(Y[:,0] - pred[:,0], Y[:,1] - pred[:,1])
    err_px = err_rel*20
    # plots:
    plt.figure('test_model()', clear=False)
    plt.subplot(211)
    plt.plot((Y[:,0]-0.5)*20, (Y[:,1]-0.5)*20, '.', label='truth')
    plt.plot((pred[:,0]-0.5)*20, (pred[:,1]-0.5)*20, 'o', alpha=0.2, label='prediction')
    plt.axis('scaled')
    plt.xlabel('px')
    plt.ylabel('px')
    plt.subplot(212)
    plt.hist(err_px, 20, alpha=0.3)
    plt.xlabel('error (px)')
    plt.ylabel('n.points')
    plt.title(f'mean error : {np.mean(err_px):.3f} px ({np.mean(err_px)*umpp:.3f} um)', fontsize=10)
    plt.tight_layout()
    
    
test_model(model, nimgs=300, ampl=3)